<a href="https://colab.research.google.com/github/FacuRoffet99/information-theory-armonization/blob/main/Material_Complementario.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Aplicando Teoría de la Información como Evaluador de Técnicas de Armonización para Datasets Multi-sitio de rs-fMRI - Material Complementario - Código

Parte de este código fue tomado y adaptado de los siguientes repositorios:

*   [https://github.com/dagush/WholeBrain](https://github.com/dagush/WholeBrain)
*   [https://gitlab.com/cristophersfr/fisher-networks](https://gitlab.com/cristophersfr/fisher-networks)



# Configuraciones

In [ ]:
# Descargar librerías
import sys
!{sys.executable} -m pip install nilearn dask[dataframe] neuroCombat

# Armonización
from neuroCombat import neuroCombat
# Conectividad
from nilearn.connectome import ConnectivityMeasure
# Acelerar código
from numba import jit
# Manejo de paths
from pathlib import Path
# Cambio de carpeta
import os
# Medir tiempo
from timeit import default_timer as timer
# Datos
import numpy as np
import pandas as pd
import dask.dataframe
import scipy.io
# Gráficos
import matplotlib
import matplotlib.pyplot as plt
import altair as alt

     |████████████████████████████████| 9.6 MB 1.3 MB/s 
     |████████████████████████████████| 38.1 MB 1.1 MB/s 
     |████████████████████████████████| 136 kB 55.0 MB/s 
  Created wheel for neuroCombat: filename=neuroCombat-0.2.12-py3-none-any.whl size=6371 sha256=90c1d4bd63d4e327e2099894b2194b1256c466b633bbf6cda1b4d7054d33430a
  Stored in directory: /root/.cache/pip/wheels/69/cb/ed/6b85d2d01fe75ac43fff6cb93d2da82efcff02527595198ae2
Successfully built neuroCombat
  Attempting uninstall: scipy
    Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [ ]:
# Defino la carpeta de trabajo
folder = Path('')           # PONER DIRECCIÓN DE LA CARPETA MADRE
os.chdir(folder)

# Leer y procesar dataset

## Cargar datasets

Importante: ejecutar solamente el código del dataset deseado, no trabajar con más de un dataset a la vez.

### IMPAC

In [ ]:
# DESCOMENTAR Y EJECUTAR SOLAMENTE LA PRIMERA VEZ

# Descargo todas las parcelaciones
# from download_data import fetch_fmri_time_series
# fetch_fmri_time_series()

In [ ]:
# Función para obtener los datos
def _read_data_impac(path, filename):
    subject_id = pd.read_csv(os.path.join(path, 'data', filename), header=None)
    # read the list of the subjects
    df_participants = pd.read_csv(os.path.join(path, 'data',
                                               'participants.csv'),
                                  index_col=0)
    df_participants.columns = ['participants_' + col
                               for col in df_participants.columns]
    # load the structural and functional MRI data
    df_anatomy = pd.read_csv(os.path.join(path, 'data', 'anatomy.csv'),
                             index_col=0)
    df_anatomy.columns = ['anatomy_' + col
                          for col in df_anatomy.columns]
    df_fmri = pd.read_csv(os.path.join(path, 'data', 'fmri_filename.csv'),
                          index_col=0)
    df_fmri.columns = ['fmri_' + col
                       for col in df_fmri.columns]
    # load the QC for structural and functional MRI data
    df_anatomy_qc = pd.read_csv(os.path.join(path, 'data', 'anatomy_qc.csv'),
                                index_col=0)
    df_fmri_qc = pd.read_csv(os.path.join(path, 'data', 'fmri_qc.csv'),
                             index_col=0)
    df_fmri_tr = pd.read_csv(os.path.join(path, 'data',
                                          'fmri_repetition_time.csv'),
                             index_col=0)
    # rename the columns for the QC to have distinct names
    df_anatomy_qc = df_anatomy_qc.rename(columns={"select": "anatomy_select"})
    df_fmri_qc = df_fmri_qc.rename(columns={"select": "fmri_select"})

    X = pd.concat([df_participants, df_anatomy, df_anatomy_qc, df_fmri,
                   df_fmri_qc, df_fmri_tr], axis=1)
    X = X.loc[subject_id[0]]
    y = X['participants_asd']
    X = X.drop('participants_asd', axis=1)

    return X, y.values

In [ ]:
# Obtener los datos
data_train, labels_train = _read_data_impac(folder/'autism-master/autism-master','train.csv')
data_test, labels_test = _read_data_impac(folder/'autism-master/autism-master','test.csv')

### ABIDE

In [ ]:
# DESCOMENTAR Y EJECUTAR SOLAMENTE LA PRIMERA VEZ

# Descargar datos
# !python download_abide_preproc.py -d rois_aal -p dparsf -s nofilt_noglobal -o 'abide'

In [ ]:
# Funciones para obtener los datos
def _read_data_abide(path):
    # read the list of the subjects
    df_data = pd.read_csv(path)
    
    y = df_data['DX_GROUP']
    X = df_data.drop('DX_GROUP', axis=1)

    return X, y

In [ ]:
# Obtener los datos
data_train, labels_train = _read_data_abide(folder/'abide/data.csv')

### SRPBS

In [ ]:
# PARA PODER OBTENER ESTE DATASET, PRIMERO SE DEBE PEDIR PERMISO A LOS AUTORES EN https://bicr.atr.jp/dcn/en/download/harmonization/. 
# LUEGO, PROCESAR EN MATLAB Y GUARDAR LOS RESULTADOS EN /FOLDER/SRPBS/COR_SRPBS_ALL_ORG.mat

In [ ]:
# Funciones para obtener los datos
def _read_data_srpbs(path):
    # read the list of the subjects
    data = np.array(scipy.io.loadmat(path)['DATA_SUB'])
    df_data = pd.DataFrame(data, columns=['DX_GROUP', 'SITE_ID', 'AGE_AT_SCAN', 'SEX'])

    y = df_data['DX_GROUP']
    X = df_data.drop('DX_GROUP', axis=1)

    return X, y

In [ ]:
# Obtener los datos
data_train, labels_train = _read_data_srpbs(folder/'SRPBS/COR_SRPBS_ALL_ORG.mat')

## Conectividad

### Inicialización y filtrado

In [ ]:
TR = 2.                           # sampling interval
k = 2                             # 2nd order butterworth filter
flp = 0.04                        # lowpass frequency of filter
fhi = 0.07                        # highpass
from scipy.signal import butter, detrend, filtfilt
import numpy.matlib as mtlib

def demean(x,dim=0):
    dims = x.size
    return x - mtlib.tile(np.mean(x,dim), dims)  # repmat(np.mean(x,dim),dimrep)

In [ ]:
# Aplicar filtro pasabanda
def BandPassFilter(boldSignal, removeStrongArtefacts=True):
    # Convenience method to apply a filter (always the same one) to all areas in a BOLD signal. For a single,
    # isolated area evaluation, better use the method below.
    (N, Tmax) = boldSignal.shape
    fnq = 1./(2.*TR)              # Nyquist frequency
    Wn = [flp/fnq, fhi/fnq]                                   # butterworth bandpass non-dimensional frequency
    bfilt, afilt = butter(k,Wn, btype='band', analog=False)   # construct the filter
    # bfilt = bfilt_afilt[0]; afilt = bfilt_afilt[1]  # numba doesn't like unpacking...
    signal_filt = np.zeros(boldSignal.shape)
    for seed in range(N):
        if not np.isnan(boldSignal[seed, :]).any():  # No problems, go ahead!!!
            ts = demean(detrend(boldSignal[seed, :]))  # Probably, we do not need to demean here, detrend already does the job...

            if removeStrongArtefacts:
                ts[ts>3.*np.std(ts)] = 3.*np.std(ts)    # Remove strong artefacts
                ts[ts<-3.*np.std(ts)] = -3.*np.std(ts)  # Remove strong artefacts

            signal_filt[seed,:] = filtfilt(bfilt, afilt, ts, padlen=3*(max(len(bfilt),len(afilt))-1))  # Band pass filter. padlen modified to get the same result as in Matlab
        else:  # We've found problems, mark this region as "problematic", to say the least...
            warnings.warn(f'############ Warning!!! BandPassFilter: NAN found at region {seed} ############')
            signal_filt[seed,0] = np.nan
    return signal_filt

### PhaseInteractionMatrix

In [ ]:
def tril_indices_column(N, k=0):
    row_i, col_i = np.nonzero(
        np.tril(np.ones(N), k=k).T)  # Matlab works in column-major order, while Numpy works in row-major.
    Isubdiag = (col_i,
                row_i)  # Thus, I have to do this little trick: Transpose, generate the indices, and then "transpose" again...
    return Isubdiag

@jit(nopython=True)
def adif(a, b):
    if np.abs(a - b) > np.pi:
        c = 2 * np.pi - np.abs(a - b)
    else:
        c = np.abs(a - b)
    return c

In [ ]:
from scipy import signal

discardOffset = 10

@jit(nopython=True)
def FastFor(phases, N, Tmax, dFC, PhIntMatr):
  T = np.arange(discardOffset, Tmax - discardOffset + 1)
  for t in T:
    for i in range(N):
      for j in range(i+1):                    
        dFC[i, j] = np.cos(adif(phases[i, t - 1], phases[j, t - 1]))
        dFC[j, i] = dFC[i, j]
    PhIntMatr[t - discardOffset] = dFC 
  return PhIntMatr

def PhaseInteractionMatrix(ts, applyFilters = True):  # Compute the Phase-Interaction Matrix of an input BOLD signal
    (N, Tmax) = ts.shape
    npattmax = Tmax - (2*discardOffset-1)  # calculates the size of phfcd matrix

    if not np.isnan(ts).any():  # No problems, go ahead!!!
        # Data structures we are going to need...
        phases = np.zeros((N, Tmax))
        dFC = np.zeros((N, N))
        # PhIntMatr = np.zeros((npattmax, int(N * (N - 1) / 2)))  # The int() is not needed, but... (see above)
        PhIntMatr = np.zeros((npattmax, N, N))
        # syncdata = np.zeros(npattmax)

        # Filters seem to be always applied...
        ts_filt = BandPassFilter(ts)  # zero phase filter the data
        # ts_filt = ts
        for n in range(N):
            Xanalytic = signal.hilbert(demean(ts_filt[n, :]))
            phases[n, :] = np.angle(Xanalytic)

        a = FastFor(phases, N, Tmax, dFC, PhIntMatr)
        PhIntMatr = a            

    else:
        warnings.warn('############ Warning!!! PhaseInteractionMatrix.from_fMRI: NAN found ############')
        PhIntMatr = np.array([np.nan])
    return PhIntMatr

In [ ]:
# Cálculo de una matriz de correlación
def obtener_matriz(file, sep=','):

  start = timer()

  # Leer los archivos
  time_series = np.array(dask.dataframe.read_csv(file, sep=sep))

  # Trasponer la matriz porque está al revés (el tiempo tiene que estar en horizontal)
  ts = time_series.transpose()

  # Matriz de interacción de fase
  correlation_matrix = np.mean(PhaseInteractionMatrix(ts, applyFilters = True), axis=0)  

  end = timer()
  print(end - start)  

  return correlation_matrix

### Procesado

#### IMPAC

In [ ]:
sub_path = 'autism-master/autism-master'

In [ ]:
# Elijo a los sujetos
subjects_data = [data_train.iloc[i] for i in range(len(data_train))]

In [ ]:
# Elijo un atlas y obtengo las matrices de correlación
correlation_matrices = list()
for i in range(0, len(subjects_data)):

  # DESCOMENTAR EL ATLAS A UTILIZAR

  # mat = obtener_matriz(folder/sub_path/subjects_data[i]['fmri_msdl'])     #39
  # mat = obtener_matriz(folder/sub_path/subjects_data[i]['fmri_harvard_oxford_cort_prob_2mm']) #48
  # mat = obtener_matriz(folder/sub_path/subjects_data[i]['fmri_basc064'])  #64
  # mat = obtener_matriz(folder/sub_path/subjects_data[i]['fmri_basc122'])  #122
  # mat = obtener_matriz(folder/sub_path/subjects_data[i]['fmri_basc197'])  #197
  # mat = obtener_matriz(folder/sub_path/subjects_data[i]['fmri_craddock_scorr_mean'])  #249
  mat = obtener_matriz(folder/sub_path/subjects_data[i]['fmri_power_2011']) #264



  print(i)
  correlation_matrices.append(mat)

correlation_matrices = np.array(correlation_matrices)

0.5444417010003235
0
0.49401745299837785
1
0.4814688580008806
2
0.5125824169990665
3
0.5443250389998866
4
0.49642107599902374
5
0.5524924310011556
6
0.5206452909988002
7
0.8705244930006302
8
1.2324797700002819
9
1.092259577000732
10
1.124957591000566
11
1.1867620880002505
12
0.9964382870002737
13
1.0184784899993247
14
1.069424194000021
15
1.0586745760010672
16
2.3575325410001824
17
1.20823743100118
18
1.0949334300003102
19
1.2980574190005427
20
1.293735168999774
21
0.9267218789991603
22
0.941543934000947
23
1.0160197179993702
24
1.0603676950013323
25
1.487399217001439
26
0.9452620020001632
27
1.2662444220004545
28
1.1455866180003795
29
1.2630064510012744
30
0.9751996649993089
31
0.9545515250010794
32
1.6779875969987188
33
1.104130230998635
34
0.9285340799997357
35
0.9289350599992758
36
1.0203587949999928
37
0.8847003530008806
38
0.9014648630000011
39
0.9349162840007921
40
0.9118599360008375
41
1.023164779999206
42
1.4886850869988848
43
1.3049175560008734
44
1.2682508649995725
45
0.9096

#### ABIDE

In [ ]:
sub_path = 'abide/Outputs/dparsf/nofilt_noglobal/rois_aal'

In [ ]:
# Elijo a los sujetos y descarto a los que dan problemas
subjects_data = list()
drop = list()
for i in range(len(data_train)):


  # DESCOMENTAR EL ATLAS A UTILIZAR

  # filename = data_train.iloc[i]['FILE_ID']+'_rois_cc200.1D'
  filename = data_train.iloc[i]['FILE_ID']+'_rois_aal.1D'



  complete_path = folder/sub_path/filename
  if complete_path.is_file():
    subjects_data.append(data_train.iloc[i])
  else:
    drop.append(i)

data_train = data_train.drop(drop)
labels_train = labels_train.drop(drop)

In [ ]:
# Elijo un atlas y obtengo las matrices de correlación
correlation_matrices = list()
for i in range(0, len(subjects_data)):


  # DESCOMENTAR EL ATLAS A UTILIZAR


  # filename = subjects_data[i]['FILE_ID']+'_rois_cc200.1D'
  filename = subjects_data[i]['FILE_ID']+'_rois_aal.1D'


  
  complete_path = folder/sub_path/filename
  mat = obtener_matriz(complete_path,'\t') 
  print(i)
  correlation_matrices.append(mat)

correlation_matrices = np.array(correlation_matrices)

0.3631706599999234
0
0.3280486920002659
1
0.37225536000005377
2
0.3613359399996625
3
0.36400066900023376
4
0.37753431599958276
5
0.34610562899979413
6
0.41015316900029575
7
0.35957400200004486
8
0.317492952000066
9
0.3207511380005599
10
0.3510915880005996
11
0.3540811670000039
12
0.3839343250001548
13
0.306280381999386
14
0.341524782000306
15
0.386283612000625
16
0.35118645500006096
17
0.3837003670005288
18
0.3734256360003201
19
0.3578848649995052
20
0.48997618799967313
21
0.44886055100050726
22
0.3523856989995693
23
0.3474142330005634
24
0.4028485860008004
25
0.3429371660004108
26
0.39996483199956856
27
0.3646232589999272
28
0.33881195799949637
29
0.360030144999655
30
0.33345728299991606
31
0.3693062779993852
32
0.38145658200028265
33
0.34984466200057796
34
0.3663552600000912
35
0.33482235399969795
36
0.3326885989999937
37
0.37536457199985307
38
0.3332049509999706
39
0.35208017399963865
40
0.3721198559996992
41
0.5315345589997378
42
0.37545708299967373
43
0.356887939000444
44
0.393077

#### SRPBS

In [ ]:
# Convertir matriz triangular a matriz simétrica
def tri2mat(tri):

  # Bhaskara de L=n*(n-1)/2
  n = int((1+np.sqrt(1+8*len(tri)))/2)

  mat = np.zeros((n,n))
  mat[np.triu_indices(mat.shape[0], k = 1)] = tri
  mat = mat + mat.T

  # Poner unos en la diagonal
  np.fill_diagonal(mat,1)

  return mat

In [ ]:
# Elijo a los sujetos
subjects_data = [data_train.iloc[i] for i in range(len(data_train))]

In [ ]:
# Leer los datos originales y los armonizados
correlation_vectors = np.array(scipy.io.loadmat(folder/'SRPBS/COR_SRPBS_ALL_ORG.mat')['X'])
correlation_vectors_har = np.array(scipy.io.loadmat(folder/'SRPBS/COR_SRPBS_ALL_SubtractMeasurementBias.mat')['X'])
# Obtener matrices de correlación
correlation_matrices = np.apply_along_axis(tri2mat, 1, correlation_vectors)
correlation_matrices_har = np.apply_along_axis(tri2mat, 1, correlation_vectors_har)

# Armonización - ComBat

In [ ]:
# Convierto las matrices a vectores
s = correlation_matrices.shape
v = correlation_matrices.reshape((s[0],s[1]**2))

# Las filas son features y las columnas son sujetos
data = v.T



# DESCOMENTAR SOLO LA VERSIÓN CORRESPONDIENTE AL DATASET SELECCIONADO

# IMPAC
# Especificar el site y las variables biológicas a conservar
# covars = {'site': np.array(data_train['participants_site']),
#           'age': np.array(data_train['participants_age']),
#           'sex': np.array(data_train['participants_sex']), 
#           'diagnosis': np.array(labels_train)}
# covars = pd.DataFrame(covars)  

# ABIDE y SRPBS
# Especificar el site y las variables biológicas a conservar
covars = {'site': np.array(data_train['SITE_ID']),
          'age': np.array(data_train['AGE_AT_SCAN']),
          'sex': np.array(data_train['SEX']),
          'diagnosis': np.array(labels_train)} 
covars = pd.DataFrame(covars) 





# Especificar variables categoricas
categorical_cols = ['sex', 'diagnosis']
# Especificar variable del site
batch_col = 'site'

# Armonización
data_combat = neuroCombat(dat=data,
    covars=covars,
    batch_col=batch_col,
    categorical_cols=categorical_cols)["data"]

# Recuperar forma original
v = data_combat.T
correlation_matrices = v.reshape((v.shape[0], int(np.sqrt(v.shape[1])), int(np.sqrt(v.shape[1]))))

[neuroCombat] Creating design matrix
[neuroCombat] Standardizing data across features
[neuroCombat] Fitting L/S model and finding priors
[neuroCombat] Finding parametric adjustments
[neuroCombat] Final adjustment of data


# Teoría de la Información (plano Shannon-Fisher)

## Cálculos (en R)

In [ ]:
# Librería que permite ejecutar código en R
%load_ext rpy2.ipython

In [ ]:
%%R

# Entropía de Shannon 'H'
shannon_entropy <- function(probs, normalized=FALSE)
{
  # Considerar solo valores mayores a cero
  p = which(probs > 1e-30)
  
  # Cálculo de entropía
  entropy = -sum(probs[p]*log(probs[p]))
  
  # Normalización
  if(normalized)
  {
    entropy = entropy/log(length(probs))
  }
  
  return(entropy)
}

# Información de Fisher 'F'
fisher_information <- function(probs)
{
  # Cantidad de nodos
  N = length(probs)
  
  # the fisher information
  f = 1/2*sum((sqrt(probs[2:N]) - sqrt(probs[1:(N-1)]))^2)
  
  return(f)
}

In [ ]:
%%R

# Evaluar una red
EvaluateNetwork <- function(graph){

  # Simplificar grafo
  if(!is_simple(graph)){
    graph <- simplify(graph)  
  }
  
  # Cantidad de nodos
  N <- vcount(graph)

  # Resultados originales
  # adjacencyMatrix <- as_adj(graph)

  # corrMatOrder
  # A <- as_adj(graph, sparse = TRUE)
  # (order.AOE <- corrMatOrder(A, order = "AOE"))
  # adjacencyMatrix <- A[order.AOE,order.AOE]
  
  # Convertir a matriz de adjacencia
  A <- as_adj(graph, attr = "weight")
  x <- as.matrix(A)
  
  # Ordenar la matriz de manera óptima
  orow <- seriate(dist(x), method ="OLO")
  A <- seriation::permute(x, ser_permutation(orow, orow))  
  adjacencyMatrix <- Matrix(A)
  # print(adjacencyMatrix)
  
  if(isSymmetric(adjacencyMatrix) == FALSE){
    stop("Graph is not undirected")
  }
  
  # Métricas adicionales
  p <- edge_density(graph) # Density
  M <- ecount(graph) # Number of Edges 
  L <- mean_distance(graph) # Average path length
  CC <- transitivity(graph) # Clustering Coefficient
  K <- mean(degree(graph)) # Average Degree
  # fit <- fit_power_law(degree(graph)) # Fit power law 
  
  # Obtener H y F de cada nodo
  results <- parRapply(cl, adjacencyMatrix,  function(m_row){ 
    row <- rep(x = 0, N)
    neighbors <- which(m_row > 0)  
    k <- sum(m_row)
    if(k > 0){
      row[neighbors] <- m_row[neighbors]/k
    }     

    # H y F
    entropy <- shannon_entropy(row)
    fisher <- fisher_information(row)   
    results <- cbind(entropy, fisher)

    return(results)
  })
  
  results <- matrix(results, nrow = 2, ncol = N)
  entropy <- results[1,]
  fisher <- results[2,]

  # H y F de la red
  H <- 1/(N*log(N - 1))*sum(entropy)
  H <- round(H, 3)
  FIM <- round(mean(fisher),3)
  
  result <- c(N, M, K, p, L, CC, -1, -1, -1, -1, -1, H, FIM)
  
  return(result)
}

## Librerías (en R)

In [ ]:
%%R
install.packages("igraph")
install.packages("doParallel")
install.packages("seriation")

library(igraph)
library(Matrix)
library(parallel)
library(doParallel)
library(seriation)

##### Configuring parallelization #####
nCluster <- detectCores() - 2
cl <-makeCluster(2, type="FORK")
registerDoParallel(cl)

R[write to console]: Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

R[write to console]: trying URL 'https://cran.rstudio.com/src/contrib/igraph_1.2.11.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 2398028 bytes (2.3 MB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[wri

## Evaluación

In [ ]:
# Transformación de las matrices de correlación en matrices de adjacencia
@jit
def corr2adj(corr):

  # Cantidad de sujetos y parcelas
  n_subjects = len(corr)
  # Crear lista vacía
  adj = list()
  # Obtener la matriz de adjacencia para cada sujeto
  for i in range(0, n_subjects): 
    intervals = corr[i].shape[0]
    adj_mat = np.zeros((intervals, intervals))
    for x in range(0, intervals):
      for y in range(0, intervals):

        if abs(corr[i][x,y]) > 0.5:
          # adj_mat[x,y] = abs(corr[i][x,y])
          adj_mat[x,y] = 1
        else:
          adj_mat[x,y] = 0        
    adj.append(adj_mat) 

  return adj 

# Calcular las métricas de las redes
def get_shannon_fisher(adj):

  %R -i adj  
  %R graph <- graph_from_adjacency_matrix(adj, mode = "undirected", weighted = TRUE)  
  %R results <- EvaluateNetwork(graph)
  %R -o results
  # Quedarse solo con H y F
  shannon_fisher = np.array([float(results[11]), float(results[12])])
  
  return shannon_fisher

In [ ]:
# Cantidad de sujetos
n_subjects = len(data_train)


# DESCOMENTAR LA SEGUNDA LÍNEA SOLO CUANDO SE TRABAJA CON SRPBS ARMONIZADO CON DVS

adjacency_matrices = np.array(corr2adj(np.array(correlation_matrices)))
# adjacency_matrices = np.array(corr2adj(np.array(correlation_matrices_har)))


shannon_fisher = list()
for i in range(0, n_subjects):
  sh = list()
  sh.append(get_shannon_fisher(adjacency_matrices[i]))
  shannon_fisher.append(sh)
  print(i)
shannon_fisher = np.array(shannon_fisher)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

## Guardado y carga de datos

In [ ]:
s_f = shannon_fisher[:,0,:]
df = pd.DataFrame(s_f, columns=['shannon', 'fisher'])

site = list()
for i in range(0, len(subjects_data)):



  # DESCOMENTAR SOLO LA VERSIÓN CORRESPONDIENTE AL DATASET SELECCIONADO

  # ABIDE
  s = subjects_data[i][5]   

  # IMPAC y SRPBS
  # s = subjects_data[i][0]


  site.append(s)  

# df["diagnosis"] = labels_train
df["site"] = site

In [ ]:
# Guardar dataframe
df.to_csv(folder/'Resultados/img/14_ Dataframes/abide-aal-combat.csv', index=False)

In [ ]:
# Cargar dataframe
df = pd.read_csv(folder/'Resultados/img/14_ Dataframes/srpbs-traveling.csv')

# Gráficos

In [ ]:
# Crear un gráfico a partir de un dataframe

selection = alt.selection_multi(fields=['site'])
color = alt.condition(selection,
                      alt.Color('site:N', legend=None, scale=alt.Scale(scheme='category20')),
                      alt.value('white'))

scatter = alt.Chart(df).mark_circle(size=80).encode(
    x=alt.X('shannon', title='Entropía de Red Normalizada'),
    y=alt.Y('fisher', title='Información de Fisher de Red Normalizada'),
    # scale=alt.Scale(domain=(0.2,0.74))
    color=color,
    opacity=alt.value(1),
    fillOpacity=alt.condition(selection, alt.value(1), alt.value(0)),
    stroke=alt.value('black'),
    strokeWidth=alt.value(0.7),
    strokeOpacity=alt.condition(selection, alt.value(1), alt.value(0.2))
).properties(
    width=650,
    height=650
).interactive()

legend = alt.Chart(df).mark_circle(size=80).encode(
    y=alt.Y('site:N', axis=alt.Axis(orient='right'), title='Sitio de adquisición'),
    color=color,
    stroke=alt.value('black'),
    strokeWidth=alt.value(0.3)
).add_selection(
    selection
)

alt.hconcat(scatter, legend, center=True).configure_axis(
    labelFontSize=15,
    titleFontSize=25
)

alt.HConcatChart(...)

# Análisis cuantitativo

In [ ]:
from scipy.stats import kruskal
from scipy.stats import f_oneway

In [ ]:
def get_metrics(df, neg_log=False):
  sha = list()
  fis = list()
  for s in df.site.unique():
    val_s = df[df['site']==s]['shannon'].values
    val_f = df[df['site']==s]['fisher'].values
    sha.append(val_s)
    fis.append(val_f)
  return -np.log10([kruskal(*sha).pvalue, kruskal(*fis).pvalue])

def print_metrics(ds, har, p_s, p_f, atlas=None):
  print(20*'*', ds, 20*'*')
  if atlas != None:
    print('Atlas: ', atlas)
  print('Method: ', har)
  print('p-values: ',p_s,p_f)
  print('')
  return

In [ ]:
met = list()
# IMPAC
df = pd.read_csv(folder/'Resultados/img/14_ Dataframes/impac-msdl.csv')
m = get_metrics(df)
met.append(m)
print_metrics('IMPAC', 'No harmonization', *m, 'MSDL')

df = pd.read_csv(folder/'Resultados/img/14_ Dataframes/impac-msdl-combat.csv')
m = get_metrics(df)
met.append(m)
print_metrics('IMPAC', 'ComBat', *m, 'MSDL')

df = pd.read_csv(folder/'Resultados/img/14_ Dataframes/impac-power.csv')
m = get_metrics(df)
met.append(m)
print_metrics('IMPAC', 'No harmonization', *m, 'Power')

df = pd.read_csv(folder/'Resultados/img/14_ Dataframes/impac-power-combat.csv')
m = get_metrics(df)
met.append(m)
print_metrics('IMPAC', 'ComBat', *m, 'Power')

******************** IMPAC ********************
Atlas:  MSDL
Method:  No harmonization
p-values:  131.94946357991662 24.793642270354358

******************** IMPAC ********************
Atlas:  MSDL
Method:  ComBat
p-values:  0.24184238573648592 4.586435328924532

******************** IMPAC ********************
Atlas:  Power
Method:  No harmonization
p-values:  178.50313955811035 136.23053092714179

******************** IMPAC ********************
Atlas:  Power
Method:  ComBat
p-values:  6.156361582694811 53.021365066346156



In [ ]:
# ABIDE
df = pd.read_csv(folder/'Resultados/img/14_ Dataframes/abide-aal.csv')
m = get_metrics(df)
met.append(m)
print_metrics('ABIDE', 'No harmonization', *m, 'AAL')

df = pd.read_csv(folder/'Resultados/img/14_ Dataframes/abide-aal-combat.csv')
m = get_metrics(df)
met.append(m)
print_metrics('ABIDE', 'ComBat', *m, 'AAL')

df = pd.read_csv(folder/'Resultados/img/14_ Dataframes/abide-craddock.csv')
m = get_metrics(df)
met.append(m)
print_metrics('ABIDE', 'No harmonization', *m, 'Craddock')

df = pd.read_csv(folder/'Resultados/img/14_ Dataframes/abide-craddock-combat.csv')
m = get_metrics(df)
met.append(m)
print_metrics('ABIDE', 'ComBat', *m, 'Craddock')

******************** ABIDE ********************
Atlas:  AAL
Method:  No harmonization
p-values:  29.131300300223952 19.49027491420303

******************** ABIDE ********************
Atlas:  AAL
Method:  ComBat
p-values:  0.008697427627224034 2.3545537628813618

******************** ABIDE ********************
Atlas:  Craddock
Method:  No harmonization
p-values:  41.584695205019266 34.4433707655307

******************** ABIDE ********************
Atlas:  Craddock
Method:  ComBat
p-values:  0.2575955224042766 7.527392061581657



In [ ]:
# SRPBS
df = pd.read_csv(folder/'Resultados/img/14_ Dataframes/srpbs.csv')
m = get_metrics(df)
print_metrics('SRPBS', 'No harmonization', *m)

df = pd.read_csv(folder/'Resultados/img/14_ Dataframes/srpbs-traveling.csv')
m = get_metrics(df)
print_metrics('SRPBS', 'Traveling dataset', *m)

df = pd.read_csv(folder/'Resultados/img/14_ Dataframes/srpbs-combat.csv')
m = get_metrics(df)
print_metrics('SRPBS', 'ComBat', *m)

******************** SRPBS ********************
Method:  No harmonization
p-values:  77.116069565349 27.69965579943464

******************** SRPBS ********************
Method:  Traveling dataset
p-values:  72.1493576437605 18.049448441218377

******************** SRPBS ********************
Method:  ComBat
p-values:  0.2853962051700903 4.3488121768735715

